In [2]:
'''
Initializing all pre-requisite packages.
'''
from automata.fa.nfa import NFA
from visual_automata.fa.nfa import VisualNFA
from tkinter import *
from tkinter import font as tkFont
import  tkinter as tk
from PIL import ImageTk, Image
from pdf2image import convert_from_path
import os, copy

'''
Initializing class to define a NonDterministic Finite Automaton.
'''
class NFA:
    #Constructor class
    def __init__(self):
        self.init = "q0" #initalState(str)
        
        self.final = set()  #finalState(set)
        
        self.trans = {}  # dict with key = previousState (number) containing a dict as value
                         #      with key = act 
                         #           val = set of poststates (numbers)
    
    
    #To accept the action based transition or not.
    def accept(self,strList):
        if len(strList) == 0:
            if self.init in self.final:
                ans = "accepted"
            else:
                ans = "rejected"
        else:
            stateList = [self.init]
            ans = "rejected"
            for state in self.steps(strList, stateList):
                if state in self.final:
                    ans = "accepted"
                    break
        return ans
    
    # To iterate over each state in the input list.
    def steps(self, strList, stateList):
        if len(strList) == 1:
            temp = self.step(strList[0],stateList)
        elif len(strList) > 1:
            symbol = strList[0]
            tail = strList[1:]
            temp = self.steps(tail, self.step(symbol,stateList))
        return(temp)
    
    # To check if the path exist to terminal state by appending all child state of the current state considered.  
    def step(self, symbol, stateList):
        temp = []
        for state in stateList:
            if state in self.trans:
                if symbol in self.trans[state]:
                    for next in self.trans[state][symbol]:
                        temp.append(next)
        return(temp)
    
    #To accept the State based transition or not.
    def acceptState(self,strList):
        temp = []
        tempStore = []
        ans = "rejected"
        for i in strList:
            for key, value in self.stateD.items():
                if i == str(value):
                    temp.append(key)
        for i in range(len(temp)):
            start = temp[0]
            upcome = temp[1:]
            if start in self.trans:
                for value in self.trans[start].values():
                    try:
                        if (value.pop()) == upcome[0]:
                            temp = upcome
                    except:
                        break
        if self.final == set(temp):
            ans = "accepted"
        return ans
    
    # Converting the finite automaton into its S-word counterpart.
    def make_sfa(self,label):
        temp = sfa()
        temp.labels = { label }
        invStateD = { self.init : 0 }
        istate = 1
        for i in self.trans: #iterate over prevState
            if not(i in invStateD):
                invStateD[i] = { istate }
                istate += istate 
            temp.trans[invStateD[i]] = {}
            poststates = set()
            for act in self.trans[i]: #iterate over actions
                temp.actD[s2s({act})] ={ act }
                for post in self.trans[i][act]: #iterate over postState 
                    if not(post in invStateD):
                        invStateD[post] = istate
                        istate += istate        
                    poststates.add(invStateD[post])
                temp.trans[invStateD[i]] = dicMerge(temp.trans[invStateD[i]],
                                                    { s2s({act}): poststates })
        for i in invStateD:
            temp.stateD[invStateD[i]] = { label : i }
            if i in self.final:
                temp.final.add(invStateD[i])
        return temp

'''
Initializing Child class sfa of parent class NFA.
'''    
    # sfa = set/structured finite automata (for S-words)
    # nfa with trans given for states and acts described by dictionaries 
    # stateD decoding/structuring state as a dictionary [record]
    # actD decoding str(act) [better: s2s(act)] to expose the set behind act
    # plus
    # labels used by stateD to describe states
    # make disjoint from other sfa to keep information private
    # and method accS for accept given a list of sets (internally representing
    #                                   a set qua symbol as the string from s2s)

class sfa(NFA):

#  init = 0
#  stateD = {}
#  actD = {}
#  labels = { "need to fix for use in stateD" }                 
    
    # Initating the Constructor function.
    def __init__(self):
        super().__init__()
        self.init = 0
        self.final = set()
        self.stateD = { 0:{} }
        self.actD = {}
        self.labels = { "need to fix for use in stateD" }                 
    
    # To check for the acceptance of actions performed in the automaton.
    def accS(self,listSet):
        return self.accept(sfaInput(listSet))
    
    def acc(self,listSet):
        return self.accept(sfaInput(listSet))
    
    # To check for the acceptance of sequence of states in the automaton.
    def mapper(self,st):
        return self.acceptState(stateInput(st))  
    
    def voc(self):
        temp = set()
        for a in self.actD:
            temp.update(self.actD[a])
        return temp

# To convert input listSet of state representation to string format.
def stateInput(listSet):
    temp = []
    for i in listSet:
        temp.append(str(i))
    return temp

# To convert input listset of action representation to string format.
def sfaInput(listSet):
    temp = []
    for i in listSet:
        temp.append(s2s(i))
    return temp
    
# The variant of str(set0) s.t. s2s({-1,-3,-2}) == s2s({-2,-3,-1})
def s2s(set0):
    temp = []
    for i in set0:
        temp.append(i)
    temp.sort()
    return str(temp)

# To merge two dict s-word automaton representation.
def dicMerge(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in dic1:
            temp[key] = temp[key].union(dic2[key])
        else:
            temp[key] = dic2[key]
    return temp
    
# To encode interval a in action (public) but not in state (private)
def uld(a):   
    temp = NFA()
    temp.init = "u"      # + "(" + str(a) + ")"
    temp.final = {"d"}   # + "(" +str(a) + ")"}
    la = "l"             # +"("+str(a)+")"
    temp.trans = { temp.init: {a: {la}},
                   la: {-a: temp.final}  }
    return temp

'''
Superposing two definite automaton of s-words.
'''

# Main function definition to superpose two automatons.
def sup(sfa1,sfa2,voc1,voc2):     
    temp = sfa()
    temp.stateD = { 0: {1:sfa1.init, 2:sfa2.init} }
    newp = {(sfa1.init,sfa2.init)}
    done = set()
    while not(newp==set()):
        nextp = set()
        for q in newp:
            morecu = rulecu(sfa1,sfa2,voc1,voc2,q,temp)
            mored1 = ruled1(sfa1,voc2,q,temp)
            mored2 = ruled2(sfa2,voc1,q,temp) 
            morep = morecu.union(mored1.union(mored2))
            nextp = nextp.union(morep)
        done = done.union(newp)
        newp = nextp.difference(done)
        for i in temp.stateD:
            if temp.stateD[i][1] in sfa1.final and temp.stateD[i][2] in sfa2.final:
                temp.final.add(i)
    temp.stateD = unwindSD(temp.stateD,sfa1.stateD,sfa2.stateD)
    temp.labels = sfa1.labels.union(sfa2.labels)
    return temp

#  To unwind temp.stateD using sfa1, sfa2
def unwindSD(sD,sD1,sD2):
    temp = {}
    for i in sD:
    # flatten:        
        temp[i] = dicUnion(sD1[sD[i][1]],sD2[sD[i][2]])
    return temp

# To perform union of two dictionary.
def dicUnion(dic1,dic2):
    temp = dic1.copy()
    for key in dic2:
        if key in temp:
            if not(dic2[key] == temp[key]):
                print("Clashing keys/labels (dicUnion)")
        temp[key] = dic2[key]
    return temp

# To convert string to list
def listify(arg):
    return arg if isinstance(arg, list) else [arg]

# To add cu to p [side-effects on temp]        
def rulecu(sfa1,sfa2,voc1,voc2,q,temp): 
    morep = set()
    moretrans = {}
    if (q[0] in sfa1.trans  and  q[1] in sfa2.trans):
        for a1 in sfa1.trans[q[0]]:
            for a2 in sfa2.trans[q[1]]:
                s1 = sfa1.actD[a1]
                s2 = sfa2.actD[a2]
                s = s1.union(s2)
                if contain(s1,voc2,s2) and contain(s2,voc1,s1):
                    for r1 in sfa1.trans[q[0]][a1]:
                        for r2 in sfa2.trans[q[1]][a2]:
                            r = (r1,r2)
                            morep.add(r)
                            temp.actD[s2s(s)] = s
                            add2stateD(temp,r)
                            qcode = decodeState(temp,q)
                            di = {s2s(s): {decodeState(temp,r)}} 
                            if qcode in temp.trans:
                                temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                             di)
                            else:
                                temp.trans[qcode] = di
    return morep

# To add d1 to p [side-effects on temp]
def ruled1(sfa1,voc2,q,temp): 
    morep = set()
    moretrans = {}
    if q[0] in sfa1.trans:
        for a1 in sfa1.trans[q[0]]:
            s = sfa1.actD[a1]
            if contain(s,voc2,set()):
                for r1 in sfa1.trans[q[0]][a1]:
                    r = (r1,q[1])
                    morep.add(r)
                    temp.actD[s2s(s)] = s
                    add2stateD(temp,r)
                    qcode = decodeState(temp,q)
                    di = {s2s(s): {decodeState(temp,r)}} 
                    if qcode in temp.trans:
                         temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                      di)
                    else:
                         temp.trans[qcode] = di
    return morep

# To add d2 to p [side-effects on temp]
def ruled2(sfa2,voc1,q,temp): 
    morep = set()
    moretrans = {}
    if q[1] in sfa2.trans:
        for a2 in sfa2.trans[q[1]]:
            s = sfa2.actD[a2]
            if contain(s,voc1,set()):
                for r2 in sfa2.trans[q[1]][a2]:
                    r = (q[0],r2)
                    morep.add(r)
                    temp.actD[s2s(s)] = s
                    add2stateD(temp,r)
                    qcode = decodeState(temp,q)
                    di = {s2s(s): {decodeState(temp,r)}} 
                    if qcode in temp.trans:
                         temp.trans[qcode] = dicMerge(temp.trans[qcode],
                                                      di)
                    else:
                         temp.trans[qcode] = di
    return morep

# To add visited states to statedD.
def add2stateD(sfa,r):
    old = False
    for i in sfa.stateD:
        if sfa.stateD[i]=={1:r[0], 2:r[1]}:
            old = True
    if old == False:
        sfa.stateD[len(sfa.stateD)] = {1:r[0], 2:r[1]}

# To decode state values.
def decodeState(sfa,r):
    found = -1
    for i in sfa.stateD:
        if sfa.stateD[i]=={1:r[0], 2:r[1]}:
            found = i
    return found

# To check for a subset of a set.
def contain(set1,set2,set3):
    return set1.intersection(set2).issubset(set3)


'''
Superposing greater than or equal to 2 automaton of s-words.
'''
# To define the main calling function.
def supULD(n):
    temp = uld(n).make_sfa(n)
    if n == 1:
        return temp
    elif 1<n:
        return sup(temp,supULD(n-1),voc(n),vocA(n-1))

# To get all the vocabularies based on the given noOfEvents >= 1. 
def voc(n):
    return {n,-n}

# Recursive function definition to get the union of all vocabularies based on noOfEvents >= 1.
def vocA(n):
    temp = voc(n)
    if n == 1:
        return temp
    elif 1<n:
        return temp.union(vocA(n-1))   
    

'''
Initializing functions responsible for visualization.
Graphical user interface build upon superposing functionalities.
'''

# To construct a dictionary to store actions and destinations possible for all possible state in the superposed automaton.
def transition(dict1,dict2,end):
    x = end.pop()
    tempDict = dict1.copy()
    for i in dict1.keys():
        for j in dict2.keys():
            if i == j:
                tempDict[str(dict2[j])] = tempDict[i]
                del tempDict[i]
    tempDict[str(dict2[x])] = {'.' : {str(dict2[x])}}
    return tempDict

# To construct a dictionary to store actions and destinations possible for all possible state in the sWord automaton.
def transitionSwords(dict1,dict2,end):
    x = end.pop()
    tempDict = dict1.copy()
    visited = set()
    for i in dict1.keys():
        for j in dict2.keys():
            if j not in visited:
                visited.add(j)
                tempDict[str(dict2[j])] = tempDict[i]
                del tempDict[i]
                break
    tempDict[str(dict2[x])] = {'[]' : {str(dict2[x])}}
    return tempDict

# To construct a set consisting of all possible actions within the automaton defined.
def actions(dict):
    temp = set()
    for i in dict.keys():
        for j in dict[i].keys():
            if j not in temp:
                temp.add(j)
    return temp

# To construct a set consisting of all possible states within the automaton defined.
def state(dict):
    temp = set()
    for i in dict.keys():
        temp.add(i)
    return temp

# To replace special character state representatio of ':' to '-'
def convertStateD(dict):
    temp = ()
    dtemp = {}
    for i,j in dict.items():
        temp = str(j)
        dtemp[i] = temp.replace(":", '-')
    return dtemp

# To change state representation of <action:State> pair within the superposed tranState dictionary. 
def finalTransition(dict1, dict2):
    tempDict = copy.deepcopy(dict1)
    for i in dict1.keys():
        for j in dict1[i].keys():
            for k in dict2.keys():
                if dict1[i][j] == set([k]):
                    #dict1[i][j] = {str(dict2[k])}
                    tempDict[i][j] = {str(dict2[k])}
    return tempDict

# To change state representation of <action:State> pair within the sWord transtate dictionary. 
def finalTransitionSwords(dict1, dict2):
    visited = set()
    l = 1
    tempDict = copy.deepcopy(dict1)
    for i in dict1.keys():
        for j in dict1[i].keys():
            for k in list(dict2.values())[l:]:
                if k not in visited:
                    visited.add(k)
                    tempDict[i]['[]'] = listify(i)
                    tempDict[i][j] = listify(k)
                    l=l+1
                    break
            break
    return tempDict

# To get the inital state of the automaton
def initial(dict):
    temp = str(dict[0])
    return temp

# get the terminal state of the automaton.
def finale(dict,end):
    temp = str(dict[end.pop()])
    return set(temp)

# To delete the terminal state.
def removeState(dict1,dict2,end):
    x = end.pop()
    temp = copy.deepcopy(dict1)
    del temp[str(dict2[x])]
    return temp

'''
Clear Frame
'''
def clearCanvas(frame):
    for widget in frame.winfo_children():
        widget.destroy()

'''
Convert the pdf into jpg format 
''' 
def pdfConvertor():
    pages = convert_from_path('C:/Users/joshu/Digraph.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton.jpg', 'JPEG')

def pdfConvertor1():
    pages = convert_from_path('C:/Users/joshu/automaton1.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton1.jpg', 'JPEG')

def pdfConvertor2():
    pages = convert_from_path('C:/Users/joshu/automaton2.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton2.jpg', 'JPEG')

def pdfConvertor3():
    pages = convert_from_path('C:/Users/joshu/automaton3.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton3.jpg', 'JPEG')

def pdfConvertor4():
    pages = convert_from_path('C:/Users/joshu/automaton4.gv.pdf', 500)
    for page in pages:
        page.save('C:/Users/joshu/automaton4.jpg', 'JPEG')

'''
Display states and transitions for event 1. 
'''        
def showAutomaton1(root):
    clearCanvas(root)
    n = eventNumber1.get()
    Label(root, text = "ULD for interval 1 from uld(1):").place(x = 0, y = 25)  #label
    Label(root, text = "Here u: Unborn l: Living & d:Dead").place(x = 0, y = 50)
    fa1 = uld(n)
    Label(root, text = "Inital Sate:").place(x = 0, y = 90)
    Label(root, text = fa1.init).place(x = 100, y = 90)  #label 
    Label(root, text = "Final Sate:").place(x = 0, y = 110)
    Label(root, text = fa1.final).place(x=100, y=110)
    Label(root, text = "Transitions:").place(x = 0, y = 130)
    Label(root, text = fa1.trans).place(x=100, y=130)
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton1(fr, fa1)).place(x=0, y=150)

'''
Display states and transitions for event 2. 
''' 
def showAutomaton2(root):
    n = eventNumber2.get()
    Label(root, text = "ULD for interval 2 from uld(2):").place(x = 600, y = 25)  #label
    Label(root, text = "Here u: Unborn l: Living & d:Dead").place(x = 600, y = 50)
    fa2 = uld(n)
    Label(root, text = "Inital Sate:").place(x = 600, y = 90)
    Label(root, text = fa2.init).place(x = 700, y = 90)  #label 
    Label(root, text = "Final Sate:").place(x = 600, y = 110)
    Label(root, text = fa2.final).place(x=700, y=110)
    Label(root, text = "Transitions:").place(x = 600, y = 130)
    Label(root, text = fa2.trans).place(x=700, y=130)
    Wdisplay1 = Button(root, text="Display the Sword automaton", command= lambda: displayAutomaton2(fr, fa2)).place(x=600, y=150)
    
'''
Display automaton for event 1. 
''' 
def displayAutomaton1(root, fa1):
    global img1
    n = eventNumber1.get()
    f1 = fa1.make_sfa(n)
    temp = copy.deepcopy(f1.trans)
    end = f1.final.copy()
    end1 = f1.final.copy()
    node = convertStateD(f1.stateD) 
    tranState = transitionSwords(temp,node,end)
    nf_1 = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState),
    transitions=finalTransitionSwords(tranState,node),
    initial_state=node[0],
    final_states={node[2]},
    )
    nf1 = VisualNFA(nf_1)
    nf1.show_diagram().render('automaton1.gv', view=True)
    pdfConvertor1()
    Label(root, text = "The Transition Diagram", font="Calibri 15").place(x=0, y=175)
    try:
        image = Image.open("C:/Users/joshu/automaton1.jpg")
        image = image.resize((500, 150), Image.ANTIALIAS)
        img1 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel1 = tk.Label(root, image = img1).place(x=10, y=205)
    panel1.pack()

'''
Display automaton for event 2. 
'''
def displayAutomaton2(root, fa2):
    global img2
    n = eventNumber2.get()
    f2 = fa2.make_sfa(n)
    temp = copy.deepcopy(f2.trans)
    end = f2.final.copy()
    end1 = f2.final.copy()
    node = convertStateD(f2.stateD) 
    tranState = transitionSwords(temp,node,end)
    nf_2 = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState),
    transitions=finalTransitionSwords(tranState,node),
    initial_state=node[0],
    final_states={node[2]},
    )
    nf2 = VisualNFA(nf_2)
    nf2.show_diagram().render('automaton2.gv', view=True)
    pdfConvertor2()
    Label(root, text = "The Transition Diagram", font="Calibri 15").place(x=600, y=175)
    try:
        image = Image.open("C:/Users/joshu/automaton2.jpg")
        image = image.resize((500, 150), Image.ANTIALIAS)
        img2 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel2 = tk.Label(root, image = img2).place(x=600, y=205)
    panel2.pack()

'''
Display superposition automaton for 2 events. 
'''
def displaySuperposition(root):
    global img
    n1 = eventNumber1.get()
    n2 = eventNumber2.get()
    fa1 = uld(n1)
    f1 = fa1.make_sfa(n1) 
    fa2 = uld(n2)
    f2 = fa2.make_sfa(n2) 
    fa = sup(f1,f2,voc(1),voc(2))
    temp = copy.deepcopy(fa.trans)
    end = fa.final.copy()
    end1 = fa.final.copy()
    l = fa.final.copy()
    x = l.pop()
    node = convertStateD(fa.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nfa = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nfa)
    nfn.show_diagram(view=True)
    pdfConvertor()
    Label(root, text = "The Transition Diagram").place(x=0, y=0)
    try:
        image = Image.open("C:/Users/joshu/automaton.jpg")
        image = image.resize((800, 350), Image.ANTIALIAS)
        img = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel3 = tk.Label(root, image = img).place(x=10, y=20)
    panel3.pack()

'''
Checking states accepted in automaton or not. 
'''
def acceptanceStateCheck(root):
    temp = []
    n1 = eventNumber1.get()
    n2 = eventNumber2.get()
    f1 = uld(n1)
    f1 = f1.make_sfa(n1) 
    f2 = uld(n2)
    f2 = f2.make_sfa(n2) 
    fa = sup(f1,f2,voc(1),voc(2))
    temp.extend(transState.get().split("."))
    ans = fa.mapper(temp)
    Label(root, text = ans, font="Calibri 10").place(x=200, y=610)

'''
Superposing multiple events. 
'''

def showSuperposeIntervals(root):
    newWindow1 = Toplevel(root)
    newWindow1.title("Automaton for more than two events")
    newWindow1.geometry("1500x1000+30+30")
    Label(newWindow1, text = "ENTER THE TOTAL NUMBER OF EVENTS:[GREATER THAN 1]", font="Calibri 15").place(x=5, y=10)
    Entry(newWindow1, textvariable = noOfEvents, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 40)#entry textbox
    Label(newWindow1, text = "ENTER THE START STATE:", font="Calibri 15").place(x=5, y=80)
    Entry(newWindow1, textvariable = startState , bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 110)#entry textbox
    button1 = Button(newWindow1, text="Dispay Automaton", command= lambda: displaySuperposeIntervals(newWindow1)).place(x=5, y = 140)
    Label(newWindow1, text = "ENTER THE TRANSITION PATH", font="Calibri 15").place(x=700, y=10)
    Entry(newWindow1, textvariable = transPath, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=700, y = 40)#entry textbox
    button1 = Button(newWindow1, text="Dispay Acceptance", command= lambda: displayPathAcceptance(newWindow1)).place(x=700, y = 60)

# To construct a subset of States and its associated <action:childState> pair.
def subState(dict1,currentState, tempStorage):
    for i in currentState:            
        if str(i) in tempStorage.keys():
            currentState.remove(i)
            continue
        else:  
            try:
                tempStorage[str(i)] = dict1[str(i)]
                for j in dict1[str(i)].values():
                    currentState.extend(j)
            except:
                currentState.remove(i)
                break
    if len(currentState) != 0:
        subState(dict1,currentState, tempStorage)
    return tempStorage    
    
'''
Display superposed intervals. 
'''
                     
def displaySuperposeIntervals(root):                     
    global img3  
    tempStorage = {}
    n = noOfEvents.get()
    curNode = startState.get()
    s1 = listify(curNode)
    fn = supULD(n) 
    temp = copy.deepcopy(fn.trans)
    end = fn.final.copy()
    end1 = fn.final.copy()
    l = fn.final.copy()
    x = l.pop()
    node = convertStateD(fn.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    temp1 = subState(finalTransition(tranState1,node), s1, tempStorage)
    allStates = state(temp1)
    allStates.add(node[x])
    nf_n = VisualNFA(
    states= allStates,
    input_symbols=actions(tranState1),
    transitions=temp1,
    initial_state=curNode,
    final_states={node[x]},
    )
    nfn = VisualNFA(nf_n)
    nfn.show_diagram().render('automaton3.gv', view=True)
    pdfConvertor3()
    Label(root, text = "The Transition Diagram").place(x=5, y=170)
    try:
        image = Image.open("C:/Users/joshu/automaton3.jpg")
        image = image.resize((600, 600), Image.ANTIALIAS)
        img3 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel4 = tk.Label(root, image = img3).place(x=10, y=200)
    panel4.pack()

'''
Highlight the accepted path over superposed intervals. 
'''    
def displayPathAcceptance(root):
    global img4 
    n = noOfEvents.get()
    tempPath = []
    fn = supULD(n) 
    temp = copy.deepcopy(fn.trans)
    end = fn.final.copy()
    end1 = fn.final.copy()
    l = fn.final.copy()
    x = l.pop()
    tempPath.extend(transPath.get().split("."))
    node = convertStateD(fn.stateD) 
    tranState = transition(temp,node,end)
    tranState1 = removeState(tranState,node,end1)
    nf_n = VisualNFA(
    states= state(tranState),
    input_symbols=actions(tranState1),
    transitions=finalTransition(tranState1,node),
    initial_state=node[0],
    final_states={node[x]},
    )
    nfn = VisualNFA(nf_n)
    nfn.show_diagram(tempPath).render('automaton4.gv', view=True)
    pdfConvertor4()
    Label(root, text = "The Acceptance Path Diagram").place(x=700, y=90)
    try:
        image = Image.open("C:/Users/joshu/automaton4.jpg")
        image = image.resize((600, 600), Image.ANTIALIAS)
        img4 = ImageTk.PhotoImage(image)
    except:
        print('Issue in drawing automaton')
    panel5 = tk.Label(root, image = img4).place(x=750, y=150)
    panel5.pack()
    
        
'''
Initializing tkinter and canvas
'''
root = Tk()
root.geometry("1500x1000+30+30")
root.title("Defining & Superposiing two interval Automaton based on {U, L, D}")
canvas = Canvas(root)
eventNumber1 = IntVar()
eventNumber2 = IntVar()
transState = StringVar()
transPath = StringVar()
noOfEvents = IntVar()
startState = StringVar()

Label(root, text="ENTER THE EVENT NUMBER:", font="Calibri 12").place(x=5, y=0)  #label
Label(root, text="FIRST:").grid(row = 0, padx=(10, 10), pady = (25,0))  #label
Label(root, text="SECOND:").grid(row = 1, padx=(10, 10), pady = (25,0))  #label
Entry(root, textvariable = eventNumber1, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=0, column=3, pady = (25,0)) #entry textbox
Entry(root, textvariable = eventNumber2, bg="LightSteelBlue1", borderwidth=1, relief="solid").grid(row=1, column=3, pady = (25,0)) #entry textbox

fr = Frame(root, height=365, width=1350, highlightbackground="blue", highlightthickness=1)
fr.pack_propagate(0) # don't shrink
fr.place(x=90, y=100)

WSignUp1 = Button(root, text="First event ULD", command= lambda: showAutomaton1(fr)).grid(row=0, column=5, padx=(10, 10), sticky=W)
WSignUp2 = Button(root, text="Second event ULD", command= lambda: showAutomaton2(fr)).grid(row=1, column=5, padx=(10, 10), sticky=W)
button = Button(root, text="Clear", command= lambda: clearCanvas(fr)).grid(row=3, column=0, padx=(10, 10),  sticky=W)

Label(root, text="SUPERPOSING THE TWO INTERVALS:", font="Calibri 12").place(x=10, y=500)  #label
fr1 = Frame(root, height=500, width=1140, highlightbackground="blue", highlightthickness=1)
fr1.pack_propagate(0) # don't shrink
fr1.place(x=300, y=470)
button1 = Button(root, text="Display Superposed automaton of event1 and event2", command= lambda: displaySuperposition(fr1)).place(x=5, y = 520)
Label(root, text="Enter State Record Sequence").place(x = 5, y = 550)   #label
Entry(root, textvariable = transState, bg="LightSteelBlue1", borderwidth=1, relief="solid").place(x=5, y = 580)#entry textbox
WSignUp1 = Button(root, text="Check for Acceptance", command = lambda: acceptanceStateCheck(root)).place(x=5, y = 610)
button1 = Button(root, text="Clear", command= lambda: clearCanvas(fr1)).place(x=5, y = 645)
button1 = Button(root, text="Superposing more than 1 events", command= lambda: showSuperposeIntervals(root)).place(x=5, y = 675)
root.mainloop()
{"mode":"full","isActive":False}

{'mode': 'full', 'isActive': False}